In [1]:
from urllib.request import urlopen, urlretrieve
from bs4 import BeautifulSoup
import pandas as pd
import re
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
import plotly.express as px
import seaborn as sns

## Pegando os gêneros 

In [2]:
df_generos = pd.read_excel("generos.xlsx")

In [3]:
df_generos

,genero,genero adj
0,Action,action
1,Adventure,adventure
2,Animation,animation
3,Biography,biography
4,Comedy,comedy
5,Crime,crime
6,Documentary,documentary
7,Drama,drama
8,Family,family
9,Fantasy,fantasy


In [4]:
lista_generos = list(df_generos["genero adj"])

In [5]:
lista_generos

['action',
 'adventure',
 'animation',
 'biography',
 'comedy',
 'crime',
 'documentary',
 'drama',
 'family',
 'fantasy',
 'film-noir',
 'history',
 'horror',
 'music',
 'musical',
 'mystery',
 'romance',
 'sci-fi',
 'short',
 'sport',
 'superhero',
 'thriller',
 'war',
 'western']

## Popular Movies by Genre

In [ ]:
# Obtendo o HTML
response = urlopen('https://www.imdb.com/search/title/?title_type=feature&genres=action&start=1&explore=genres&ref_=adv_nxt')
html = response.read().decode('utf-8')
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
paginas = " ".join(soup.find("div", class_ = 'desc').get_text().split())

In [ ]:
paginas

In [ ]:
pegando_pagina = re.findall(r"([\d]+[\.\,][\d]+)", paginas)
print(pegando_pagina)
pagina_final = int(pegando_pagina[0].replace(",", ""))
print(pagina_final)

In [ ]:
pagina_final

In [ ]:
for pagina in range(1, pagina_final, 50):
    if pagina < 10000:
        print(pagina)

In [ ]:
# Obtendo o HTML
response = urlopen('https://www.imdb.com/search/title/?title_type=feature&genres=action&start=9951&explore=genres&ref_=adv_nxt')
html = response.read().decode('utf-8')
soup = BeautifulSoup(html, 'html.parser')

###  Montando toda a estrutura de scrapping

In [ ]:
lista_generos

In [6]:
lista_dados_total = list()


for genero in lista_generos:
    
    for pagina in range(1, 100000, 50):
        
        cont = 0
    
        if pagina < 10000:
            print(f"Página: {pagina}" )
            print(f"Gênero: {genero.strip()}" )
            
            
            try:
                response = urlopen(f'https://www.imdb.com/search/title/?title_type=feature&genres={genero.strip()}&start={pagina}&explore=genres&ref_=adv_nxt')
                html = response.read().decode('utf-8')
                soup = BeautifulSoup(html, 'html.parser')
                print(f'https://www.imdb.com/search/title/?title_type=feature&genres={genero.strip()}&start={pagina}&explore=genres&ref_=adv_nxt')
                
            except:
                print(f">>>>>>>>>>>>>>> ERRO <<<<<<<<<<<<<<<" )
                print(f">>>>>>>>>>>>>>> Página: {pagina}<<<<<<<<<<<<<<<" )
                print(f">>>>>>>>>>>>>>> Gênero: {genero.strip()}<<<<<<<<<<<<<<<" )
                continue
            
            for informacoes in soup.findAll('div', class_ = "lister-item-content"):
                dict_filme = dict()
                cont = cont + 1
                
                if cont == 1:
                    print("chegou até aqui")
                    print("=======================================")

                ranking = informacoes.find("span", class_ = "lister-item-index unbold text-primary").get_text().replace(".", "")
                #print(ranking)
                dict_filme["ranking"] = ranking

                titulo = informacoes.find('a').get_text()
                #print(titulo)
                dict_filme["titulo"] = titulo

                try:
                    pegar_ano = informacoes.find("span", class_ = "lister-item-year text-muted unbold").get_text()

                    ano_extract = re.findall(r"\d+", pegar_ano)
                    dict_filme["ano_filme"] = ano_extract[0]
                except:
                    dict_filme["ano_filme"] = ""
                    #print("erro na data do filme")

                ### Duração 
                try:
                    duracao = informacoes.find("span", class_ = 'runtime').get_text()
                    minutos =  re.findall(r"\d+", duracao)
                    dict_filme["duracao"] = minutos[0]
                except:
                    dict_filme["duracao"] = ""
                    #print("erro na duração do filme")

                ### Genero 
                
                try:

                    dict_filme["genero_primario"] = " ".join(informacoes.find("span", class_ = 'genre').get_text().split()).split(",")[0]
                    dict_filme["generos_secundarios"] = " ".join(informacoes.find("span", class_ = 'genre').get_text().split()).split(",")[1:]
                    dict_filme["generos"] = " ".join(informacoes.find("span", class_ = 'genre').get_text().split()).split(",")
                
                except:
                    
                    dict_filme["genero_primario"] = ""
                    dict_filme["generos_secundarios"] = ""
                    dict_filme["generos"] = ""
                

                ### Nota 

                try:
                    dict_filme["nota"] = float(informacoes.find("strong").get_text())
                except:
                    dict_filme["nota"] = ""
                    #print("erro na nota do filme")

                ### Descrição 
                try:
                    
                    dict_filme["descricao"] = informacoes.findAll("p", class_ = 'text-muted')[1].get_text().replace("\n", "")
                    
                except:
                    
                    dict_filme["descricao"] = ""
                
                ### Elenco
                
                try:
                    
                    elenco = informacoes.findAll("p")
                    elenco = " ".join(elenco[2].get_text().split())
                    #print(elenco)
                    dict_filme["diretor"] = elenco[elenco.find(":") + 1 : elenco.find("|")].strip().split(",")
                    dict_filme["estrelas"] = elenco[elenco.find(":", elenco.find(":")+1) + 1:len(elenco)].strip().split(",")
                    
                except:
                    
                    dict_filme["diretor"] = ""
                    dict_filme["estrelas"] = ""
                    
                ### Votos 
                try:
                    dict_filme["votos"] = float(informacoes.find("p", class_ = "sort-num_votes-visible").findAll("span")[1].get_text().replace(",", ""))
                except:
                    dict_filme["votos"] = ""
                    #print("erro nos votos do filme")

                try:    
                    faturamento = informacoes.find("p", class_ = "sort-num_votes-visible").findAll("span")[4]
                    faturamento_oficial = re.findall(r"([\d]+[\.\,][\d]+[\.\,][\d]+)|([\d]+[\.\,][\d])", str(faturamento))
                    dict_filme["faturamento"] = float(faturamento_oficial[0][0].replace(",", ""))
                except:
                    dict_filme["faturamento"] = ""
                    #print("erro no faturamento do filme")
                
                dict_filme["genero_link"] = genero
                    
                #print(dict_filme)

                lista_dados_total.append(dict_filme)

Página: 1
Gênero: action
https://www.imdb.com/search/title/?title_type=feature&genres=action&start=1&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 51
Gênero: action
https://www.imdb.com/search/title/?title_type=feature&genres=action&start=51&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 101
Gênero: action
https://www.imdb.com/search/title/?title_type=feature&genres=action&start=101&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 151
Gênero: action
https://www.imdb.com/search/title/?title_type=feature&genres=action&start=151&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 201
Gênero: action
https://www.imdb.com/search/title/?title_type=feature&genres=action&start=201&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 251
Gênero: action
https://www.imdb.com/search/title/?title_type=feature&genres=action&start=251&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 301
Gênero: action
https://www.imdb.com/search/title/?title_type=feature&genres=action&start=301&explore

https://www.imdb.com/search/title/?title_type=feature&genres=action&start=2151&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 2201
Gênero: action
https://www.imdb.com/search/title/?title_type=feature&genres=action&start=2201&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 2251
Gênero: action
https://www.imdb.com/search/title/?title_type=feature&genres=action&start=2251&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 2301
Gênero: action
https://www.imdb.com/search/title/?title_type=feature&genres=action&start=2301&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 2351
Gênero: action
https://www.imdb.com/search/title/?title_type=feature&genres=action&start=2351&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 2401
Gênero: action
https://www.imdb.com/search/title/?title_type=feature&genres=action&start=2401&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 2451
Gênero: action
https://www.imdb.com/search/title/?title_type=feature&genres=action&start=2451&explore=genres&

https://www.imdb.com/search/title/?title_type=feature&genres=action&start=4301&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 4351
Gênero: action
https://www.imdb.com/search/title/?title_type=feature&genres=action&start=4351&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 4401
Gênero: action
https://www.imdb.com/search/title/?title_type=feature&genres=action&start=4401&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 4451
Gênero: action
https://www.imdb.com/search/title/?title_type=feature&genres=action&start=4451&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 4501
Gênero: action
https://www.imdb.com/search/title/?title_type=feature&genres=action&start=4501&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 4551
Gênero: action
https://www.imdb.com/search/title/?title_type=feature&genres=action&start=4551&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 4601
Gênero: action
https://www.imdb.com/search/title/?title_type=feature&genres=action&start=4601&explore=genres&

https://www.imdb.com/search/title/?title_type=feature&genres=action&start=6451&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 6501
Gênero: action
https://www.imdb.com/search/title/?title_type=feature&genres=action&start=6501&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 6551
Gênero: action
https://www.imdb.com/search/title/?title_type=feature&genres=action&start=6551&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 6601
Gênero: action
https://www.imdb.com/search/title/?title_type=feature&genres=action&start=6601&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 6651
Gênero: action
https://www.imdb.com/search/title/?title_type=feature&genres=action&start=6651&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 6701
Gênero: action
https://www.imdb.com/search/title/?title_type=feature&genres=action&start=6701&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 6751
Gênero: action
https://www.imdb.com/search/title/?title_type=feature&genres=action&start=6751&explore=genres&

https://www.imdb.com/search/title/?title_type=feature&genres=action&start=8601&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 8651
Gênero: action
https://www.imdb.com/search/title/?title_type=feature&genres=action&start=8651&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 8701
Gênero: action
https://www.imdb.com/search/title/?title_type=feature&genres=action&start=8701&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 8751
Gênero: action
https://www.imdb.com/search/title/?title_type=feature&genres=action&start=8751&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 8801
Gênero: action
https://www.imdb.com/search/title/?title_type=feature&genres=action&start=8801&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 8851
Gênero: action
https://www.imdb.com/search/title/?title_type=feature&genres=action&start=8851&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 8901
Gênero: action
https://www.imdb.com/search/title/?title_type=feature&genres=action&start=8901&explore=genres&

...

https://www.imdb.com/search/title/?title_type=feature&genres=western&start=251&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 301
Gênero: western
https://www.imdb.com/search/title/?title_type=feature&genres=western&start=301&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 351
Gênero: western
https://www.imdb.com/search/title/?title_type=feature&genres=western&start=351&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 401
Gênero: western
https://www.imdb.com/search/title/?title_type=feature&genres=western&start=401&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 451
Gênero: western
https://www.imdb.com/search/title/?title_type=feature&genres=western&start=451&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 501
Gênero: western
https://www.imdb.com/search/title/?title_type=feature&genres=western&start=501&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 551
Gênero: western
https://www.imdb.com/search/title/?title_type=feature&genres=western&start=551&explore=genres&

https://www.imdb.com/search/title/?title_type=feature&genres=western&start=2401&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 2451
Gênero: western
https://www.imdb.com/search/title/?title_type=feature&genres=western&start=2451&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 2501
Gênero: western
https://www.imdb.com/search/title/?title_type=feature&genres=western&start=2501&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 2551
Gênero: western
https://www.imdb.com/search/title/?title_type=feature&genres=western&start=2551&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 2601
Gênero: western
https://www.imdb.com/search/title/?title_type=feature&genres=western&start=2601&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 2651
Gênero: western
https://www.imdb.com/search/title/?title_type=feature&genres=western&start=2651&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 2701
Gênero: western
https://www.imdb.com/search/title/?title_type=feature&genres=western&start=2701&ex

https://www.imdb.com/search/title/?title_type=feature&genres=western&start=4551&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 4601
Gênero: western
https://www.imdb.com/search/title/?title_type=feature&genres=western&start=4601&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 4651
Gênero: western
https://www.imdb.com/search/title/?title_type=feature&genres=western&start=4651&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 4701
Gênero: western
https://www.imdb.com/search/title/?title_type=feature&genres=western&start=4701&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 4751
Gênero: western
https://www.imdb.com/search/title/?title_type=feature&genres=western&start=4751&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 4801
Gênero: western
https://www.imdb.com/search/title/?title_type=feature&genres=western&start=4801&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 4851
Gênero: western
https://www.imdb.com/search/title/?title_type=feature&genres=western&start=4851&ex

https://www.imdb.com/search/title/?title_type=feature&genres=western&start=6701&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 6751
Gênero: western
https://www.imdb.com/search/title/?title_type=feature&genres=western&start=6751&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 6801
Gênero: western
https://www.imdb.com/search/title/?title_type=feature&genres=western&start=6801&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 6851
Gênero: western
https://www.imdb.com/search/title/?title_type=feature&genres=western&start=6851&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 6901
Gênero: western
https://www.imdb.com/search/title/?title_type=feature&genres=western&start=6901&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 6951
Gênero: western
https://www.imdb.com/search/title/?title_type=feature&genres=western&start=6951&explore=genres&ref_=adv_nxt
chegou até aqui
Página: 7001
Gênero: western
https://www.imdb.com/search/title/?title_type=feature&genres=western&start=7001&ex

https://www.imdb.com/search/title/?title_type=feature&genres=western&start=8851&explore=genres&ref_=adv_nxt
Página: 8901
Gênero: western
https://www.imdb.com/search/title/?title_type=feature&genres=western&start=8901&explore=genres&ref_=adv_nxt
Página: 8951
Gênero: western
https://www.imdb.com/search/title/?title_type=feature&genres=western&start=8951&explore=genres&ref_=adv_nxt
Página: 9001
Gênero: western
https://www.imdb.com/search/title/?title_type=feature&genres=western&start=9001&explore=genres&ref_=adv_nxt
Página: 9051
Gênero: western
https://www.imdb.com/search/title/?title_type=feature&genres=western&start=9051&explore=genres&ref_=adv_nxt
Página: 9101
Gênero: western
https://www.imdb.com/search/title/?title_type=feature&genres=western&start=9101&explore=genres&ref_=adv_nxt
Página: 9151
Gênero: western
https://www.imdb.com/search/title/?title_type=feature&genres=western&start=9151&explore=genres&ref_=adv_nxt
Página: 9201
Gênero: western
https://www.imdb.com/search/title/?title_

In [17]:
len(lista_dados_total)

194163

In [18]:
df_filmes_populares = pd.DataFrame(lista_dados_total)

In [19]:
df_filmes_populares

,ranking,titulo,ano_filme,duracao,genero_primario,generos_secundarios,generos,nota,descricao,diretor,estrelas,votos,faturamento,genero_link
0,1,Shang-Chi e a Lenda dos Dez Anéis,2021,132,Action,"[ Adventure, Fantasy]","[Action, Adventure, Fantasy]",8.0,"Shang-Chi, the master of unarmed weaponry-base...",[Destin Daniel Cretton],"[Simu Liu, Awkwafina, Tony Chiu-Wai Leung, ...",67200.0,,action
1,2,Matrix Resurrections,2021,,Action,[ Sci-Fi],"[Action, Sci-Fi]",,The plot is currently unknown.,[Lana Wachowski],"[Keanu Reeves, Jessica Henwick, Yahya Abdul-...",,,action
2,3,Maligno,2021,111,Action,"[ Crime, Horror]","[Action, Crime, Horror]",6.3,Madison is paralyzed by shocking visions of gr...,[James Wan],"[Annabelle Wallis, Maddie Hasson, George You...",20362.0,,action
3,4,Free Guy: Assumindo o Controle,2021,115,Action,"[ Adventure, Comedy]","[Action, Adventure, Comedy]",7.6,A bank teller discovers that he's actually an ...,[Shawn Levy],"[Ryan Reynolds, Jodie Comer, Taika Waititi, ...",53199.0,,action
4,5,O Esquadrão Suicida,2021,132,Action,"[ Adventure, Comedy]","[Action, Adventure, Comedy]",7.4,"Supervillains Harley Quinn, Bloodsport, Peacem...",[James Gunn],"[Margot Robbie, Idris Elba, John Cena, Joel...",195527.0,,action
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194158,"8,832",Cowboy,,,Western,[],[Western],,Add a Plot,[],[],,,western
194159,"8,833",O Agente Desconhecido,1936,57,Music,"[ Romance, Western]","[Music, Romance, Western]",4.8,Bob Allen in his starring debut gets a job on ...,[Spencer Gordon Bennet],"[Robert Allen, Martha Tibbetts, Harry Woods,...",24.0,,western
194160,"8,834",Romance do Sertão,1919,50,Drama,[ Western],"[Drama, Western]",,Dreamer dairyman Phineas Dobbs of Cow Hollow s...,[Arthur Rosson],"[Tom Mix, Juanita Hansen, Pat Chrisman, Spo...",,,western
194161,"8,835",Trumpin' Trouble,1926,,Western,[],[Western],,"John Lawson stakes out a gold mining claim, bu...",[Richard Thorpe],"[Jay Wilsey, Bob Fleming, Alma Rayford, Sli...",,,western
